In [ ]:
#Import a plotting libraries and a maths library 
import matplotlib.pyplot as plt
import numpy as np
from IPython import display
%matplotlib notebook

<h1>The potential</h1>
<p>We will use a classical potential to do MD here. The cell below will plot the potential function and allow you to experiment with the two parameters epsilon, the depth fof the energy well and sigma, the distance just left of the energy well, where attractive and repulsive energies are equal.</p>

<p>We also use a distance cutoff beyond which we assume the energy is zero. To avoid a discontinuity, we will shift the potential up by whatever value it has at the cutoff radius.</p>

<p><b>Note</b>: We will use unitless quantities, i.e. all energies are multiples of epsilon and all distances are multiples of sigma and we use derived also unitless quantites for everything else.</p>

In [ ]:
epsilon = 1   # Depth of energy minimum
sigma = 1     # Radius of zero crossing point
cutoff = 2    #distance cutoff

#create distance axis
r = np.linspace(0.01,3.0,num=500)

# compute unshifted potential
unshifted = 4 * epsilon *((sigma/r)**12      - (sigma/r)**6)

# compute energy at cutoff (to be subtracted in shifted potential)
Ecutoff = 4 * epsilon *((sigma/cutoff)**12 - (sigma/cutoff)**6)

#compute shifted potential and set to zero beyond cutoff
shifted   = unshifted - Ecutoff
for i in range(len(r)):
    if r[i] >= cutoff:
        shifted[i] = 0.0

#plot
fig, ax = plt.subplots(figsize=[6,6])
ax.set_title("Lennard-Jones potential")
ax.plot(r, unshifted, 'r-', linewidth=1, label="unshifted")
ax.plot(r, shifted, 'b-', linewidth=1, label="shifted")
ax.set_xlim([0.0, 3.0])
ax.set_ylim([-1.5, 1.5])
ax.set_ylabel("Energy/epsilon")
ax.set_xlabel("r/sigma")
ax.axhline(0, color='grey',linestyle='--',linewidth=2)
ax.axvline(sigma, color='grey',linestyle='--',linewidth=2)
ax.axvline(cutoff, color='blue',linestyle='--',linewidth=1, label="cutoff")
l = ax.legend()

<h1>The actual MD</h1>

In [ ]:
# Set simulation parameters
DIM      = 2      # Dimensions, leave this at 2 unless you really know what's going to happen
N        = 32     # Number of particles
BoxSize  = 10.0   # Box size
NSteps   = 10000  # Number of steps
deltat   = 0.0032 # Time step in reduced time units
Ttarget  = 0.5#   # Reduced temperature
DumpFreq = 100    # Updating of results every DumFreq steps

###################################################################################
# A function to compute the acceleration, potential energy and virial coefficient
###################################################################################
def Compute_Acceleration(pos, acc, ene_pot, BoxSize, DIM, N):
    # Compute forces on positions using the Lennard-Jones potential
    # Uses double nested loop which is slow O(N^2) and unsuitable for large systems
    
    #prepare list of size DIM to hold distanced in direct and fractional coordinates
    Sij = np.zeros(DIM) # Scaled distance
    Rij = np.zeros(DIM) # Direct distance (unitless sigma coordinates)
    
    #Set all variables to zero
    ene_pot = ene_pot * 0.0
    acc = acc * 0.0
    virial = 0.0
    
    # Loop over all pairs of particles
    for i in range(N-1):
        for j in range(i+1, N): #i+1 to N ensures we do not double count
            
            # Compute distance in scaled units
            Sij = pos[i,:] - pos[j,:] 
            
            # If a distance along and DIM is larger than 0.5 in scaled units,
            # we subtract 1 to make it interact with the closest image in 
            # periodic boundary conditions. This is called minimum-image
            # convention.
            for l in range(DIM): 
                if (np.abs(Sij[l]) > 0.5):
                    Sij[l] = Sij[l] - np.copysign(1.0, Sij[l])
            
            #transform to direct distance and compute length squared
            Rij = BoxSize * Sij
            Rsqij = np.dot(Rij, Rij)
            
            #check if length is smaller than cutoff and if yes compute energy and force
            if(Rsqij < cutoff**2):
                
                #mathematically cheap way to get 1/r^6 and 1/r^12 from r^2
                rm2 = 1.0 / Rsqij
                rm6 = rm2**3.0
                rm12 = rm6**2.0
                
                #compute shifted LJ potential
                phi = epsilon * (4.0 * (rm12 - rm6) - Ecutoff)
                
                #compute force as LJ potential derivative
                dphi = epsilon * 24.0 * rm2 * (2.0 * rm12 - rm6)
                
                #accumulate energy, virial (for pressure) and acceleration (assuming mass=1)
                ene_pot[i] = ene_pot[i] + 0.5 * phi
                ene_pot[j] = ene_pot[j] + 0.5 * phi
                virial = virial + dphi*np.sqrt(Rsqij)
                acc[i,:] = acc[i,:] + dphi*Sij
                acc[j,:] = acc[j,:] - dphi*Sij
    
    return acc, np.sum(ene_pot) / N, -virial / DIM

###################################################################################
# A function to calculate the temperature
###################################################################################
def Calculate_Temperature(vel,BoxSize,DIM,N):
    
    # set kinetic energy to zero
    ene_kin = 0.0
    
    # compute velocity in direct coordinates and accumulate kinetic energy (assume mass=1)
    for i in range(N):
        direct_vel = BoxSize * vel[i,:]
        ene_kin = ene_kin + 0.5 * np.dot(direct_vel, direct_vel)
    
    #get the average kinetic energy and temperature
    ene_kin_aver = 1.0 * ene_kin / N
    temperature = 2.0 * ene_kin_aver / DIM
    
    return ene_kin_aver, temperature

###################################################################################
# Actual MD program starts here
###################################################################################
# initialize particles at random positions in scaled coordinates [-0.5, 0.5)
pos = (np.random.randn(N, DIM)-0.5)

# initialize velocities and accelerations in [-0.5, 0.5]
vel = (np.random.randn(N,DIM)-0.5)
acc = (np.random.randn(N,DIM)-0.5)

# create lists to store average kinetic and potential energy at each step
ene_kin_aver = np.ones(NSteps)
ene_pot_aver = np.ones(NSteps)
# create list to store potential energy at each step
ene_pot = np.ones(N)
# create lists to store temperature, virial and pressure at each step
temperature = np.ones(NSteps)
virial = np.ones(NSteps)
pressure = np.ones(NSteps)

#compute volume and density needed for pressure
volume  = BoxSize**DIM
density = N / volume

#prepare the plots
fig, ax = plt.subplots(ncols=2, nrows = 4, figsize=(20, 10), sharex='col')
gs = ax[0, 0].get_gridspec()
for a in ax[0:, -1]:
    a.remove()
ax_disp = fig.add_subplot(gs[0:, -1])
ax_ekin = ax[0, 0]
ax_epot = ax[1, 0]
ax_temp = ax[2, 0]
ax_pres = ax[3, 0]
ax_disp.set_aspect('equal')
ax_disp.set_xlabel('x')
ax_disp.set_ylabel('y')
ax_disp.set_xlim(-0.5*BoxSize,0.5*BoxSize)
ax_disp.set_ylim(-0.5*BoxSize,0.5*BoxSize)
ax_ekin.set_ylabel('Ekin')
ax_epot.set_ylabel('Epot')
ax_temp.set_ylabel('Temp')
ax_pres.set_xlabel('Timestep')
ax_pres.set_ylabel('Pres')
ax_pres.set_xlim((0, NSteps))
for i in range(N):
    ax_disp.plot(pos[i,0] * BoxSize, pos[i,1] * BoxSize, 'o' , markersize=20)
ekin_line, = ax_ekin.plot(ene_kin_aver[:0],'r-')
epot_line, = ax_epot.plot(ene_pot_aver[:0],'b-')
temp_line, = ax_temp.plot(temperature[:0],'g-')
pres_line, = ax_pres.plot(pressure[:0],'k-')

# a loop over all steps
for k in range(NSteps):

    # rRfold positions into [-0.5, 0.5) according to periodic boundary conditions
    for i in range(DIM):
        period = np.where(pos[:,i] > 0.5)
        pos[period,i]=pos[period,i] - 1.0
        period = np.where(pos[:,i] < -0.5)
        pos[period,i]=pos[period,i] + 1.0

    # Update positions according to velocity and acceleration
    pos = pos + deltat * vel + 0.5 * (deltat**2.0) * acc

    # Calculate temperature
    ene_kin_aver[k],temperature[k] = Calculate_Temperature(vel, BoxSize, DIM, N)

    # Rescale velocities and take half step
    chi = np.sqrt(Ttarget / temperature[k])
    vel = chi * vel + 0.5*deltat*acc

    # Compute new acceleration (=forces, assuming mass=1)
    acc, ene_pot_aver[k], virial[k] = Compute_Acceleration(pos, acc, ene_pot, BoxSize, DIM, N)

    # Take second half set in velocity update
    vel = vel + 0.5*deltat*acc

    # Calculate temperature
    ene_kin_aver[k],temperature[k] = Calculate_Temperature(vel,BoxSize,DIM,N)

    # Calculate pressure
    pressure[k]= density*temperature[k] + virial[k]/volume

    # Update output every DumpFreq number of steps
    if(k % DumpFreq == 0 and k > 200):
        if(DIM==2):
            ekin_line.set_xdata(range(k))
            ekin_line.set_ydata(ene_kin_aver[:k])
            ax_ekin.set_ylim((np.min(ene_kin_aver[100:k]), np.max(ene_kin_aver[200:k])))
            epot_line.set_xdata(range(k))
            epot_line.set_ydata(ene_pot_aver[:k])
            ax_epot.set_ylim((np.min(ene_pot_aver[100:k]), np.max(ene_pot_aver[200:k])))
            temp_line.set_xdata(range(k))
            temp_line.set_ydata(temperature[:k])
            ax_temp.set_ylim((np.min(temperature[100:k]), np.max(temperature[200:k])))
            pres_line.set_xdata(range(k))
            pres_line.set_ydata(pressure[:k])
            ax_pres.set_ylim((np.min(pressure[100:k]), np.max(pressure[200:k])))
            for i in range(N):
                ax_disp.lines[i].set_xdata(pos[i,0]*BoxSize)
                ax_disp.lines[i].set_ydata(pos[i,1]*BoxSize)
                  
            display.clear_output(wait=True)
            display.display(plt.gcf())